In [37]:
%run Requirements.ipynb

In [38]:
%run secrets.ipynb

In [39]:
df = pd.read_csv('TopGlobalArtists50.csv')
selected_column = df['ArtistName']
selected_column.to_csv('ArtistList.csv', index=False)

Get the Total Reach of Artist From Last FM After Integrating with Spotify

In [40]:
artist_names_df = pd.read_csv("ArtistList.csv")
artist_data_list = []
api_key = Secrets.get('lastFM_APIkey')

for index, row in artist_names_df.iterrows():
    artist_name = row["ArtistName"]
    
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json"
    response = requests.get(url)
    artist_info = response.json()
    artist_data = {}
    if "artist" in artist_info:
        artist = artist_info["artist"]
        artist_data["ArtistName"] = artist.get("name")
        stats = artist.get("stats", {})
        artist_data["Listeners"] = stats.get("listeners")
        artist_data["PlayCount"] = stats.get("playcount")
        tags = artist.get("tags", {}).get("tag", [])
        genres = [tag["name"] for tag in tags]
        artist_data["Genres"] = ", ".join(genres)
        artist_data_list.append(artist_data)
    else:
        print(f"No artist information found for {artist_name}.")
artist_df = pd.DataFrame(artist_data_list)

display(artist_df)


,ArtistName,Listeners,PlayCount,Genres
0,Tommy Richman,605079,5827431,"rnb, Hip-Hop, rap, post-punk, funk"
1,Sabrina Carpenter,1567085,124375753,"pop, indie pop, alternative, folk, pop rock"
2,Kendrick Lamar,3649681,534405353,"Hip-Hop, rap, seen live, west coast, compton"
3,Billie Eilish,2843988,380244127,"pop, indie pop, seen live, electropop, indie"
4,Billie Eilish,2843988,380244127,"pop, indie pop, seen live, electropop, indie"
...,...,...,...,...
58,Taylor Swift,4650581,2346109905,"country, pop, female vocalists, singer-songwri..."
59,Lord Huron,1559832,40152300,"folk, seen live, indie, psychedelic, indie pop"
60,Luis R Conriquez,70224,2243133,"mexico, latin, español, corridos, REGIONAL MEX..."
61,Neton Vega,372,5004,


Get Track wise Rreach of Artist From Last FM after integrating with Spotify

In [48]:
spotify_tracks_df = pd.read_csv("TopGlobalArtists50.csv")

def get_track_info(artist_name, track_name):
    api_key = "130ad717055f46130e0be133f8497c20"
    url = f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={api_key}&artist={artist_name}&track={track_name}&format=json"
    response = requests.get(url)
    track_info = response.json().get("track", {})
    track_id = track_info.get("mbid")
    duration = track_info.get("duration")
    listeners = track_info.get("listeners")
    play_count = track_info.get("playcount")
    album = track_info.get("album", {}).get("title")
    tags = [tag["name"] for tag in track_info.get("toptags", {}).get("tag", [])]
    
    return {
        "TrackName": track_name,
        "ArtistName": artist_name,
        "TrackID": track_id,
        "Duration": duration,
        "Listeners": listeners,
        "PlayCount": play_count,
        "Album": album,
        "Tags": ", ".join(tags)
    }

track_info_list = []

for index, row in spotify_tracks_df.iterrows():
    artist_name = row["ArtistName"]
    track_name = row["track"]
    track_info = get_track_info(artist_name, track_name)
    track_info_list.append(track_info)

track_info_df = pd.DataFrame(track_info_list)
display(track_info_df)

# Save the DataFrame to a CSV file
# track_info_df.to_csv("track_info_lastfm.csv", index=False)


,TrackName,ArtistName,TrackID,Duration,Listeners,PlayCount,Album,Tags
0,MILLION DOLLAR BABY,Tommy Richman,None,155000,554500,4018243,MILLION DOLLAR BABY,
1,Espresso,Sabrina Carpenter,None,0,757962,11400978,None,"espresso, pop, dance-pop, nu-disco, soty"
2,Not Like Us,Kendrick Lamar,None,274000,672370,5754524,Not Like Us,"MySpotigramBot, Diss, diss track, Hip-Hop, hip..."
3,LUNCH,Billie Eilish,None,179000,450837,2409206,HIT ME HARD AND SOFT,"pop rock, electropop, indie rock, electro, Love"
4,CHIHIRO,Billie Eilish,None,0,399901,2258972,None,"alt-pop, indietronica, deep house, perfect, ar..."
...,...,...,...,...,...,...,...,...
58,Guilty as Sin?,Taylor Swift,None,254000,401158,5677733,THE TORTURED POETS DEPARTMENT,"worst of 2024, chatgpt, pop, jackson mahomes, ..."
59,The Night We Met,Lord Huron,None,221000,1137012,16431254,Strange Trails,"indie, folk, indie folk, 13 reasons why, Sound..."
60,Si No Quieres No,Luis R Conriquez,None,0,11777,81813,None,
61,Si No Quieres No,Neton Vega,None,0,1,2,None,


Get the information regarding the Albums of the Artist from the Last FM after integraing with Spotify

In [50]:
spotify_albums_df = pd.read_csv("TopGlobalArtists50.csv")

def get_album_info(artist_name, album_name):
    api_key = "130ad717055f46130e0be133f8497c20"
    url = f"http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={api_key}&artist={artist_name}&album={album_name}&format=json"
    response = requests.get(url)
    
    try:
        album_info = response.json().get("album", {})
    except ValueError:
        print(f"Invalid response for artist '{artist_name}' and album '{album_name}': {response.text}")
        return None

    album_id = album_info.get("mbid")
    listeners = album_info.get("listeners")
    play_count = album_info.get("playcount")
    tags = [tag["name"] for tag in album_info.get("toptags", {}).get("tag", [])]
    release_date = album_info.get("wiki", {}).get("published")

    return {
        "AlbumName": album_name,
        "ArtistName": artist_name,
        "AlbumID": album_id,
        "Listeners": listeners,
        "PlayCount": play_count,
        "Tags": ", ".join(tags),
        "ReleaseDate": release_date
    }

album_info_list = []

for index, row in spotify_albums_df.iterrows():
    artist_name = row["ArtistName"]
    album_name = row["album"] 
    album_info = get_album_info(artist_name, album_name)
    if album_info:
        album_info_list.append(album_info)

album_info_df = pd.DataFrame(album_info_list)
display(album_info_df)
# album_info_df.to_csv("album_info_lastfm.csv", index=False)


,AlbumName,ArtistName,AlbumID,Listeners,PlayCount,Tags,ReleaseDate
0,MILLION DOLLAR BABY,Tommy Richman,48510e76-3985-4c0d-8ee0-4eff139e84d9,570923,4846004,,None
1,Espresso,Sabrina Carpenter,82156d59-0ff6-4257-a77c-55ace40d550a,754721,11288286,,None
2,Not Like Us,Kendrick Lamar,a3631980-cdac-4d12-854f-8ba8b95808c4,674739,5839531,,None
3,HIT ME HARD AND SOFT,Billie Eilish,54e8ca23-b8cf-4e2d-aedd-d70242d0c67f,543850,17014330,,"05 May 2024, 17:35"
4,HIT ME HARD AND SOFT,Billie Eilish,54e8ca23-b8cf-4e2d-aedd-d70242d0c67f,543850,17014330,,"05 May 2024, 17:35"
...,...,...,...,...,...,...,...
58,THE TORTURED POETS DEPARTMENT,Taylor Swift,3caf3d66-5b6c-4e0d-856a-af7f294ea0ff,618602,20631498,,"05 May 2024, 11:09"
59,Strange Trails,Lord Huron,05bbbba9-1ddf-401b-a41c-3d0a8dc805bd,1192482,21908152,,"03 Mar 2022, 02:49"
60,Si No Quieres No,Luis R Conriquez,,11819,82290,,None
61,Si No Quieres No,Neton Vega,,1,2,,None
